In [ ]:
import os
import re
import sys
import glob
import tables
import numpy as np
from scipy.signal import welch
from scipy.fft import fft, fftfreq
import matplotlib
import matplotlib.pyplot as plt
if '..' not in sys.path:
    sys.path.append('..')
from dlml.data import read_area_values

In [ ]:
data_folder = os.path.join('..', 'data')
network_name = 'IEEE39'
network_type = 'converted_from_PowerFactory'
stoch_loads = 'all_stoch_loads'
sub_folders = 'var_H_area_1'
suffix = 'compensator'
if suffix == 'compensator':
    H_G2, H_G3 = 4.33, 4.47
    H_G2, H_G3 = 4.432, 4.559
    H = [5, H_G2, H_G3, 3.57, 4.33, 4.35, 3.77, 3.47, 3.45, 4.2]
    file_fmt = 'inertia_' + '_'.join(map(lambda s: f'{s:.03f}', H)) + '_*_' + suffix + '.h5'
else:
    file_fmt = 'inertia_*_' + suffix + '.h5'
data_files = sorted(glob.glob(os.path.join(data_folder,
                                           network_name,
                                           network_type,
                                           stoch_loads,
                                           sub_folders,
                                           file_fmt)))
var_name = 'Vd_bus3'
gen_areas_map = [['G02','G03','Comp11'],
                 ['G04','G05','G06','G07','Comp21'],
                 ['G08','G09', 'G10','Comp31'],
                 ['G01']]
gen_Pnom = {'G01': 10000e6, 'G02': 700e6, 'G03': 800e6, 'G04': 800e6, 'G05': 300e6,
            'G06': 800e6, 'G07': 700e6, 'G08': 700e6, 'G09': 1000e6, 'G10': 1000e6,
            'Comp11': 100e6, 'Comp21': 100e6, 'Comp31': 100e6}
X = []
momentum = np.zeros(len(data_files))
for i,data_file in enumerate(data_files):
    fid = tables.open_file(data_file)
    if data_file == data_files[0]:
        time = fid.root.time.read()[:-1]
    X.append(fid.root[var_name].read()[:, :-1])
    fid.close()
    H = map(float, re.findall('\d+.\d+', data_file))
    area_ID = 1
    for h,(name,S) in zip(H, gen_Pnom.items()):
        if name in gen_areas_map[area_ID - 1]:
            momentum[i] += h * S
momentum = 2 * momentum * 1e-9 / 60
idx = np.argsort(momentum)
momentum = momentum[idx]
X = [X[i] for i in idx]

In [ ]:
N_traces, N_samples = X[0].shape
dt = np.diff(time[:2])[0]
sampling_rate = 1 / dt
minutes = int(N_samples * dt / 60)

In [ ]:
P, Pm = [], []
Xf, Xfm = [], []
freq = fftfreq(N_samples, dt)[:N_samples//2]
for x in X:
    F,p = welch(x, sampling_rate, nperseg=64*minutes)
    P.append(p)
    Pm.append(p.mean(axis=0))
    xf = fft(x)
    xf = 2.0 / N_samples * np.abs(xf[:, 0:N_samples//2])
    Xf.append(xf)
    Xfm.append(xf.mean(axis=0))

In [ ]:
units = r'GW$\cdot$s$^2$'
fig, ax = plt.subplots(2, 1, sharex=True, figsize=(8,6))
cmap = plt.get_cmap('viridis', len(Xfm))
for i,xfm in enumerate(Xfm):
    ax[0].semilogx(freq, 20 * np.log10(xfm), color=cmap(i), lw=1, label=f'{momentum[i]:.4f} {units}')
for i,pm in enumerate(Pm):
    ax[1].loglog(F, pm, color=cmap(i), lw=1)
for a in ax:
    for side in 'right','top':
        a.spines[side].set_visible(False)
    a.grid(which='major', axis='both', lw=0.5, ls=':', color=[.6,.6,.6])
ax[0].legend(loc='lower left')
ax[0].set_ylim([-30, 25])
ax[0].set_ylabel('FFT [dB]')
ax[1].set_xlabel('Frequency [Hz]')
ax[1].set_ylabel(r'PSD [V$^2$/Hz]')
fig.tight_layout()
pdf_file = os.path.splitext(file_fmt)[0] + '_' + var_name + '.pdf'
fig.savefig(pdf_file.replace('*', '_'))